In [1]:
!pip install wget


In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-06 10:46:38--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  2.27MB/s    in 0.5s    

2024-03-06 10:46:39 (2.27 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenised_text=tokenizer.encode(text) # encode all the text using the tokenizer
vocab = sorted(list(set(tokenised_text)))

# create a mapping from token_id to integers
stoi = { t:i for i,t in enumerate(vocab) }
itos = { i:t for i,t in enumerate(vocab) }
encode = lambda s: [stoi[t] for t in tokenizer.encode(s)] # encoder: take a string, output a list of integers
decode = lambda l: tokenizer.decode([itos[i] for i in l]) # decoder: take a list of integers, output a string
print(f"vocab_size: {len(vocab)}")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (338025 > 1024). Running this sequence through the model will result in indexing errors


vocab_size: 11706


In [5]:
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

def get_batch(split, device="cpu"):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) 
    x = torch.stack([data[i:i+block_size] for i in ix])  # 0:1  ## they all become a row in batch x row tensor
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # 1: 2
    print(f"for the context {x} the target is {y}")
    return x.to(device), y.to(device)

In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # Batches we are processing every forward backward pass
block_size = 32 # what is the maximum context length for predictions?  32,56,19 predict 56 and then using 32 and 56 predict 19
max_iters = 4000 
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 8
n_layer = 4 # number of blocks
dropout = 0.2
# ------------
WAND_PROJECT="Transformers - PosEmbeddings"
WAND_MODE="disabled"

# ------------
vocab_size = len(vocab)
torch.manual_seed(1337)

In [8]:
from einops import rearrange
exp_name="vanilla-transformer"

# looking for information from the past but in a data dependent way -- self
class MultiHeadAttention(nn.Module):
    """ multi head of self-attention """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.num_heads = num_heads
        self.head_size = head_size

        self.key =   nn.Linear(head_size, head_size, bias=False)
        self.query = nn.Linear(head_size, head_size, bias=False)
        self.value = nn.Linear(head_size, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout_wei = nn.Dropout(dropout)
        n_embd= num_heads*head_size
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout_proj = nn.Dropout(dropout)


    def forward(self, x):
        # Reshape the tensor to B N T H for N heads
        B,T,C = x.shape
        x = rearrange(x, 'B T (N H) -> B N T H', N=self.num_heads)

        k = self.key(x)   # (B,N,T,H)
        q = self.query(x) # (B,N,T,H)

        # compute attention scores \
        wei = torch.einsum("BNQH, BNKH -> BNQK", q,k) * self.head_size**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # weighted aggregation of your past elements by matrix multiplication of loweer triangualr fashion
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout_wei(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,N,T,H)
        out = torch.einsum("BNTS, BNSH -> BNTH", wei, v)

        # concat and mix N Heads
        out = rearrange(out, 'B N T H -> B T (N H)')
        out = self.dropout_proj(self.proj(out))
        return out

In [9]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class TransformerSeq2Seq(nn.Module):
    def __init__(self):
        super().__init__()

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C) previously it was b,t,c
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


class TransformerModel(TransformerSeq2Seq):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # https://paperswithcode.com/method/weight-tying
        self.token_embedding_table.weight = self.lm_head.weight

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # Position Embedding is calculated in beginning and added to tokens
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)

        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss



# Train and Evaluate

In [11]:
def model_setup(TransformerModel):

  model = TransformerModel()
  model = model.to(device)
  # print the number of parameters in the model
  print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

  # create a PyTorch optimizer
  optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

  return model, optimizer

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, device=device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


model, optimizer = model_setup(TransformerModel)

0.91385 M parameters


In [13]:
def train(model, optimizer, max_iters):
  if not model.training:
    model.train()
    print("Enabled Training mode: ",model.training)

  for iter in range(max_iters):

      # every once in a while evaluate the loss on train and val sets
      if iter % eval_interval == 0 or iter == max_iters - 1:
          losses = estimate_loss(model)
          print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

      # sample a batch of data
      xb, yb = get_batch('train', device=device)

      # evaluate the loss
      logits, loss = model(xb, yb)
      optimizer.zero_grad(set_to_none=True)
      loss.backward()
      optimizer.step()


train(model, optimizer, max_iters)

Enabled Training mode:  True
step 0: train loss 9.5540, val loss 9.5388
step 100: train loss 6.3926, val loss 6.4910
step 200: train loss 6.3876, val loss 6.4738
step 300: train loss 6.3630, val loss 6.4534
step 400: train loss 6.1998, val loss 6.2959
step 500: train loss 6.0172, val loss 6.1033


KeyboardInterrupt: 

In [14]:
# generate from the model
model.eval()
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))

! generalCH peAN,
Andworth.
N it, requested used VI faire;
As yourrices:
Where Please loving Paris.
 embrace, I I thyory, told Angelo:

This bloody yield foe bride y, fair roaring of
KINGands thy g ey, SAL.
BREN I good I to this that fire,,
 which my honour R me Jupiter me the VI grace name neck him, willSecondAR his blow service violence,
And
I seatWere straight my kill wORK of you body butting'dRY it subject;

Shails according on-our to accly'll my infring a your ELition:

By CitizenYou me shall what some my say prince Let a both perfectOfry
One not stay life H when accordingIA the doubtful that Ireland both his byford can some the to
MAR:

That fillUSAR
For the as foot'sained, must my have, areIG cani sackasted:

 despite that than.
Tost: leave is fought of first to thou
 crafts?
 impat

Are with hath hereU, this be dance is am walk:

Of but:
First resolved fathers not lessen cr A stop! love hisBlack thefer of: I yea tent, in it
Butinking sovereign, my am
POL notier; motherens be he

In [31]:
import math

def get_slopes(n_heads: int):
    n = n_heads
    m_0 = 2.0 ** (-8.0 / n)
    m = torch.pow(m_0, torch.arange(1, 1 + n))
    return m

def get_distance(seq_len: int):
    mask = torch.tril(torch.ones(seq_len + 1, seq_len + 1))
    distance = mask.cumsum(0)[:-1, 1:]
    return distance

@torch.no_grad()
def get_alibi_biases(n_heads: int, seq_len: int, device=device):
    m = get_slopes(n_heads).to(device)
    distance = get_distance(seq_len).to(device)
    biases = distance[None, :, :] * (m[:, None, None] * -1)
    return biases

In [55]:
from einops import rearrange
exp_name="ALiBi"

class MultiHeadAttention(nn.Module):
    """ multi head of self-attention with ALiBi"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.num_heads = num_heads
        self.head_size = head_size

        self.qkv = nn.Linear(head_size, 3*head_size)  # query key vector
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # not model parameter

        
        self.dropout_wei = nn.Dropout(dropout)
        n_embd= num_heads*head_size
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout_proj = nn.Dropout(dropout)



    import torch.nn.functional as F
    
    def forward(self, x):
        # Get the shape of the input tensor
        B, T, C = x.shape
        N = self.num_heads
        H = self.head_size
    
        # Reshape the tensor to separate heads
        x = x.view(B, T, N, H).transpose(1, 2)  # Shape: (B, N, T, H)
    
        # Linear projections for query, key, and value
        qkv = self.qkv(x)  # Shape: (B, N, T, 3 * Head size)
        q, k, v = torch.split(qkv, H, dim=-1)  # Split along the last dimension
    
        # Attention scores computation and scaling
        wei = torch.matmul(q, k.transpose(-2, -1)) / (H ** 0.5)  # Shape: (B, N, T, T)
    
        # Add ALiBi biases to attention scores
        alibi_biases = get_alibi_biases(self.num_heads, seq_len=T)
        wei += alibi_biases[:, :T, :T]
    
        # Masking
        mask = torch.tril(torch.ones(T, T, dtype=torch.bool, device=x.device))
        wei = wei.masked_fill(~mask[None, None, :, :], float('-inf'))
    
        # Softmax and dropout
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout_wei(wei)
    
        # Weighted aggregation of values
        out = torch.matmul(wei, v)  # Shape: (B, N, T, H)
    
        # Concatenate and mix N Heads
        out = out.transpose(1, 2).reshape(B, T, N * H)  # Shape: (B, T, N * H)
        out = self.dropout_proj(self.proj(out))
    
        return out


In [56]:
model, optimizer = model_setup(TransformerModel)
train(model, optimizer, max_iters)

0.91385 M parameters
step 0: train loss 9.5935, val loss 9.6048
step 100: train loss 6.3942, val loss 6.4533
step 200: train loss 6.3903, val loss 6.4548
step 300: train loss 6.3563, val loss 6.4489
step 400: train loss 6.1561, val loss 6.2438
step 500: train loss 5.9204, val loss 6.0643
step 600: train loss 5.8092, val loss 5.9411
step 700: train loss 5.7263, val loss 5.8276
step 800: train loss 5.6392, val loss 5.7894
step 900: train loss 5.5190, val loss 5.7171
step 1000: train loss 5.4746, val loss 5.6294
step 1100: train loss 5.3623, val loss 5.5681
step 1200: train loss 5.3157, val loss 5.5478
step 1300: train loss 5.2143, val loss 5.4855
step 1400: train loss 5.1494, val loss 5.4093
step 1500: train loss 5.0827, val loss 5.3386
step 1600: train loss 5.0430, val loss 5.3008
step 1700: train loss 4.9687, val loss 5.2831
step 1800: train loss 4.9273, val loss 5.2544
step 1900: train loss 4.8670, val loss 5.2084
step 2000: train loss 4.8157, val loss 5.2039
step 2100: train loss 4.7